In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import math


In [ ]:
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)


In [ ]:
def extract_landmarks(hand_landmarks, w, h):
    points = []
    for lm in hand_landmarks.landmark:
        points.append((int(lm.x * w), int(lm.y * h)))
    return points


def euclidean(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)


def draw_geometry(points, shape):
    canvas = np.zeros(shape, dtype=np.uint8)

    
    finger_pairs = [(9, 12), (13, 16), (17, 20)]
    for base, tip in finger_pairs:
        cv2.line(canvas, points[base], points[tip], 255, 3)

    return canvas


In [ ]:
circle_frames = 0
CIRCLE_STABLE_FRAMES = 5
circle_hold = 0


In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    geometry = np.zeros((h, w), dtype=np.uint8)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            points = extract_landmarks(hand_landmarks, w, h)

            
            geometry = draw_geometry(points, (h, w))

            
            thumb = points[4]
            index = points[8]

            dist = euclidean(thumb, index)
            hand_size = euclidean(points[0], points[9])
            
            CIRCLE_ENTER_RATIO = 0.25   
            CIRCLE_EXIT_RATIO  = 0.32   


            enter_condition = dist < CIRCLE_ENTER_RATIO * hand_size
            exit_condition  = dist > CIRCLE_EXIT_RATIO * hand_size

            if enter_condition:
                circle_frames += 1
            elif exit_condition:
                circle_frames = 0


            

            stable_circle = circle_frames >= CIRCLE_STABLE_FRAMES

            if stable_circle:
                circle_hold = 8
            elif circle_hold > 0:
                circle_hold -= 1

            
            if circle_hold > 0:
                center = (
                    (thumb[0] + index[0]) // 2,
                    (thumb[1] + index[1]) // 2
                )
                radius = int(0.25 * hand_size)
                cv2.circle(geometry, center, radius, 255, 2)

            # Hough Line Transform 
            lines = cv2.HoughLinesP(
                geometry,
                rho=1,
                theta=np.pi / 180,
                threshold=40,
                minLineLength=30,
                maxLineGap=10
            )

            line_count = 0 if lines is None else len(lines)

            wow = (circle_hold > 0) and (line_count >= 2)

            
            mp_draw.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS
            )

            
            if wow:
                cv2.putText(
                    frame,
                    "WOW Gesture Detected!",
                    (40, 60),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1.2,
                    (0, 255, 0),
                    3
                )

            
            cv2.putText(
                frame,
                f"Dist: {int(dist)}  Lines: {line_count}",
                (40, 100),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (255, 255, 0),
                2
            )

    cv2.imshow("WOW Gesture Recognition", frame)
    cv2.imshow("Geometry", geometry)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
